In [37]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_community.embeddings.bedrock import BedrockEmbeddings
from langchain.vectorstores.chroma import Chroma


In [3]:
DATA_PATH = "./data"

In [4]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

In [6]:
documents = load_documents()

In [28]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False
    )
    return text_splitter.split_documents(documents)

In [29]:
documents = load_documents()
chunks = split_documents(documents)

In [43]:
def get_embedding_function():
    embeddings = BedrockEmbeddings(
        credentials_profile_name="default", region_name='us-east-1'
    )
    return embeddings

In [42]:
chunks[0].metadata.get('source')

'data\\Методика ВНИИГАЗ.pdf'

In [44]:
from langchain_aws.embeddings.bedrock import BedrockEmbeddings

In [45]:
embeddings = BedrockEmbeddings(credentials_profile_name="bedrock-admin", region_name="us-east-1")

ValidationError: 1 validation error for BedrockEmbeddings
  Value error, Could not load credentials to authenticate with AWS client. Please check that credentials in the specified profile name are valid. Bedrock error: The config profile (bedrock-admin) could not be found [type=value_error, input_value={'credentials_profile_nam...gion_name': 'us-east-1'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error